In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pandas

2023-07-20 06:39:32.108400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 06:39:34.274869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
import requests
from bs4 import BeautifulSoup
import spacy
import pandas as pd

In [12]:
def fetch_article_content(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

In [13]:
def extract_person_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    person_entities = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return list(set(person_entities))

In [4]:
def main(news_article_url_text):
    news_article_url = news_article_url_text
    article_content = fetch_article_content(news_article_url)
    person_entities = extract_person_entities(article_content)

    print("Names of humans found in the article:")
    print(person_entities)

In [6]:
if __name__ == "__main__":
    news_article_url_text = "https://www.englandhockey.co.uk/media/news/u16-boys-and-girls-win-series-against-scotland"
    main(news_article_url_text)

Names of humans found in the article:
['Ollie Stone', 'Harry Montague', 'Don Vinson', 'England U16', 'Grace O’Malley-Kumar', 'Jake Holland', 'Cecily Moorhead', 'Barrett', 'Emily Barrett', 'Nottingham']


In [14]:
def process_csv(input_file, output_file):
    df = pd.read_csv(input_file)
    names_list = []

    for link in df['Links']:
        article_content = fetch_article_content(link)
        person_entities = extract_person_entities(article_content)
        names_list.append(person_entities)

    # Create new columns for each name found in the articles
    df_names = pd.DataFrame(names_list).add_prefix('Name_')
    df = pd.concat([df, df_names], axis=1)

    # Save the results to a new CSV file
    df.to_csv(output_file, index=False)

In [16]:
def main():
    input_csv_file = "Input111.csv"
    output_csv_file = "output.csv"
    process_csv(input_csv_file, output_csv_file)
    print("Names extraction completed. Results saved to 'output.csv'.")


In [17]:
if __name__ == "__main__":
    main()

Names extraction completed. Results saved to 'output.csv'.
